# Resource optimization model

** What this notebook shows...the solution for given **


Problem: Optimize the process of resource selection based on the given constraints of Cost, Level, Points and Motivation factor range.
Solution: solution is developed based on LPP modeling and monte_carlo simulation method.

Ex: this problem will help to select best x number of resources out of y number of resources given constraints. Where x<y.

## Step 1) 
Importing necessary libraies
> Creating simulation function for producing real values for motivation factor.
> Uniform dis used for selecting floating values from the specific range of motivation factor values 

Refrence: ** https://en.wikipedia.org/wiki/Simulation **

In [1]:
import pandas as pd
import numpy as np
import math
import random
from matplotlib import pyplot as plt
from IPython.display import clear_output

'''
PI = 3.1415926
e = 2.71828
'''

    
def crude_monte_carlo(number,lower_value,upper_value):
    """
    The function will performs the Crude Monte Carlo for given
    specific function f(x) on the range.
    Number represents how many time simlaution should be perfomred. 
    
    """
    lower_bound = lower_value 
    upper_bound = upper_value
    num_samples = number
    
    sum_of_samples = 0
    for i in range(num_samples):
        x = random.uniform(lower_bound, upper_bound )
        sum_of_samples += x
    return (sum_of_samples/num_samples)

In [7]:
import itertools
data1= pd.read_excel('D:\\Humber_Course\\Term-2\\Capstone Project-PMPG 5503 RNA\\Presentation\\Conceptiual Model_5.xlsx',sheet_name='Sheet3')
data1

,Level,Cost / Hour,Points / Day
0,Junior,50,0.5
1,Intermediate,90,1.0
2,Senior,140,1.5


In [8]:
data2=pd.read_excel('D:\\Humber_Course\\Term-2\\Capstone Project-PMPG 5503 RNA\\Presentation\\Conceptiual Model_5.xlsx',sheet_name='Sheet2')
data2.head()

,First Name,Last Name,Level,Motivational Factors (-),Motivational Factors (+)
0,John,Doe,Junior,0.80,1.0
1,Kevin,Worth,Junior,1.20,1.4
2,Ravi,Suppiah,Junior,0.60,0.8
3,Luke,Styles,Junior,0.78,1.1
4,Jason,Danes,Junior,1.24,1.4


In [9]:
data=pd.merge(data2, data1, how='left', on='Level')
data.head()
# combined Final Resource Pool

,First Name,Last Name,Level,Motivational Factors (-),Motivational Factors (+),Cost / Hour,Points / Day
0,John,Doe,Junior,0.80,1.0,50,0.5
1,Kevin,Worth,Junior,1.20,1.4,50,0.5
2,Ravi,Suppiah,Junior,0.60,0.8,50,0.5
3,Luke,Styles,Junior,0.78,1.1,50,0.5
4,Jason,Danes,Junior,1.24,1.4,50,0.5


In [12]:
data.describe()

,Motivational Factors (-),Motivational Factors (+),Cost / Hour,Points / Day,Motivation Factor,Actual points
count,100.00000,100.000000,100.000000,100.00000,100.000000,100.000000
mean,0.99980,1.243700,80.000000,0.85000,1.121981,0.953109
std,0.33543,0.317969,34.815531,0.39248,0.322996,0.537406
min,0.60000,0.800000,50.000000,0.50000,0.700570,0.350285
25%,0.80000,0.997500,50.000000,0.50000,0.899310,0.525623
50%,0.90500,1.170000,70.000000,0.75000,1.029638,0.852135
75%,1.08000,1.392500,90.000000,1.00000,1.197867,1.186791
max,1.93000,2.000000,140.000000,1.50000,1.965317,2.879976


## Step 2)
** Here we used
pd.Series(list(itertools.combinations(data.Resource,3)))
where number 3 define number of resources we need in each combination

In this problem we have 100 resource(person)
We need 3 resource for our project.

This step will perform all necessary calculation for calculating values for combination of 3 resources.

In [10]:
""" 
No of simlation should be fixed from developement team as it is taking time for computation for higher numbers.
Decision for simation time: upto developement team. 

Here 5000 simulation time is selected as it is enough sample size to represent population for small range of data. 
And Average is the best represent for that range which has low standard devation.
"""
iteration=5000
# itreation present no of simlation (check for which number  for if it is taking 5 second for 100000 we can show.)
data["Resource"] = data["First Name"] +" "+ data["Last Name"]
for i, row in data.iterrows():
    data.loc[i,"Motivation Factor"]= crude_monte_carlo(iteration,row['Motivational Factors (-)'],row['Motivational Factors (+)'])
    data['Actual points']=data['Motivation Factor'] * data['Points / Day']
    d1 = data.set_index('Resource').to_dict()['Cost / Hour']
    d2= data.set_index('Resource').to_dict()['Points / Day']
    d3= data.set_index('Resource').to_dict()['Motivation Factor']
    d4= data.set_index('Resource').to_dict()['Actual points']
    s = pd.Series(list(itertools.combinations(data.Resource,3)))
    v1 = s.map(lambda k: d1[k[0]] + d1[k[1]]+d1[k[2]])
    v2 = s.map(lambda k: d2[k[0]] + d2[k[1]]+d2[k[2]])
    v3 = s.map(lambda k: d3[k[0]] + d3[k[1]]+d3[k[2]])
    v4 = s.map(lambda k: d4[k[0]] + d4[k[1]]+d4[k[2]])
    df2= pd.DataFrame({'Resources': s.str.get(0)+'_' + s.str.get(1)+ '_'+s.str.get(2), 'Cost': v1,'Points/Day':v2,'Motivation Factor':v3,'Actual points':v4})
df2.head()

,Resources,Cost,Points/Day,Motivation Factor,Actual points
0,John Doe_Kevin Worth_Ravi Suppiah,150,1.5,2.901494,1.450747
1,John Doe_Kevin Worth_Luke Styles,150,1.5,3.139969,1.569985
2,John Doe_Kevin Worth_Jason Danes,150,1.5,3.519772,1.759886
3,John Doe_Kevin Worth_Emily Dean,150,1.5,3.115714,1.557857
4,John Doe_Kevin Worth_Connor White,150,1.5,3.377080,1.688540


In [15]:
#df2.describe

## Step3)

Now we have combinations (161700 combinations) of 3 resources and their repective values for cost, points/day, motivation factor(selected and calculated based on simulation) and actual poitns. 
 
Now we calculate cost per sprint and points per sprint and total cost for project for each combination.

In [16]:
p1=df2.sort_values(by=[('Actual points')],ascending=False)
p2=p1.rename(columns={'Motivation Factor': 'Motivation_Factor', 'Actual points': 'Actual_points'})

No_of_hours= 8 
# this will be fix as working hours per day is 8

Sprint_Duration = 10 #(in days)

# this functionality is given on project description page
No_of_release= 4
sprints_per_release= 6

#Cost is given in hour (table Data in cell 3) so we need to multiply it with No_of_hours and Sprint_duration to get Total_cost_per_sprint
p2['Total_cost_per_sprint']=p2['Cost'] *No_of_hours*Sprint_Duration

p2['Points_per_sprint']=p2['Actual_points'] *Sprint_Duration
p2['Cost_per_project']=p2['Total_cost_per_sprint']*No_of_release*sprints_per_release

p3=p2.head(500)
p3


,Resources,Cost,Points/Day,Motivation_Factor,Actual_points,Total_cost_per_sprint,Points_per_sprint,Cost_per_project
161019,Bavarase Lemson _Angaad Jasper_Jess Larkyn,420,4.5,5.340032,8.010048,33600,80.100484,806400
160898,Bavarase Lemson _Thav Agdal _Angaad Jasper,420,4.5,5.334518,8.001776,33600,80.017764,806400
160952,Bavarase Lemson _Viswa Chris _Angaad Jasper,420,4.5,5.183640,7.775460,33600,77.754604,806400
160982,Bavarase Lemson _Ritvik Simon _Angaad Jasper,420,4.5,5.175906,7.763859,33600,77.638590,806400
154422,Anaiya Torres _Bavarase Lemson _Angaad Jasper,370,4.0,5.799919,7.717220,29600,77.172204,710400
144266,Aryan Barswin _Bavarase Lemson _Angaad Jasper,370,4.0,5.749901,7.667202,29600,76.672021,710400
160997,Bavarase Lemson _Harshpaul Arantxa_Angaad Ja...,420,4.5,4.949225,7.423837,33600,74.238369,806400
161003,Bavarase Lemson _Kabe Diem _Angaad Jasper,420,4.5,4.927866,7.391799,33600,73.917986,806400
161139,Thav Agdal _Angaad Jasper_Jess Larkyn,420,4.5,4.925330,7.387994,33600,73.879945,806400
160899,Bavarase Lemson _Thav Agdal _Jess Larkyn,420,4.5,4.919964,7.379946,33600,73.799459,806400


In [33]:
p4 = pd.DataFrame(p3.Resources.str.split('_',2).tolist(),columns = ['Resource_1','Resource_2','Resurce_3'])
p4.head()

,Resource_1,Resource_2,Resurce_3
0,Bavarase Lemson,Angaad Jasper,Jess Larkyn
1,Bavarase Lemson,Thav Agdal,Angaad Jasper
2,Bavarase Lemson,Viswa Chris,Angaad Jasper
3,Bavarase Lemson,Ritvik Simon,Angaad Jasper
4,Anaiya Torres,Bavarase Lemson,Angaad Jasper


Step 4)

Visulizing Resource combinations 
User can see that each dot is combination of resources. It also gives idea how much total project cost, points per sprint, and motivational factor for each combination.  

In [40]:
import plotly.express as plt
plt.scatter(p3,x="Motivation_Factor",y="Points_per_sprint",color="Cost_per_project",hover_name="Resources")

In [47]:
#plt.scatter(p3,x="Cost_per_project",y="Points_per_sprint",color="Motivation_Factor",hover_name="Resources")